# 0.Download sequences from GISAID

In [ ]:
# Download sequences_fasta_2024_01_27.tar.xz and metadata_tsv_2024_01_27.tar.xz from GISAID
# Unzip the file---it takes a long time
! tar -xvJf 0.rawdata/metadata_tsv_2024_01_27.tar.xz
! tar -xvJf 0.rawdata/sequences_fasta_2024_01_27.tar.xz

# 1.Filter time range【2022.10.1-2023.7.31】and separate chinese sequences 【China】

## short sequences

In [2]:
# Filter short sequences[27000-30000]
! seqkit seq -m 27000 -M 30000 -j 30 0.rawdata/sequences.fasta >0.rawdata/seq.fil.fa

[WARN] you may switch on flag -g/--remove-gaps to remove spaces


## filter unmatched date

In [1]:
# 1.Remove sequences whose dates do not match
# 2.Remove sequences whose date format do not match
# 3.Remove chinese sequences

! mkdir 2.filter_metadata
import re

input_file = "0.rawdata/metadata.tsv"
output_file_date = "2.filter_metadata/metadata_date_unmatched.tsv"
output_file_china = "2.filter_metadata/metadata_china_unmatched.tsv"
output_file_final = "2.filter_metadata/metadata_filter_china_date.tsv"

with open(input_file, 'r') as f_in, open(output_file_china, 'w') as f_out_china, open(output_file_date, 'w') as f_out_date, open(output_file_final, 'w') as f_out_final:
    for line in f_in:
        columns = line.split('\t')
        match = re.search(r'\d{4}-\d{2}-\d{2}', columns[5]) # format match; date format[yyyy-mm-dd]
        if match:
            date = match.group(0)
            if "2022-10-01" <= date <= "2023-07-31": # date match; time range[2022.10.22-2023.7.14]
                    if "China" in columns[6]: # identify chinese seq[China]
                        f_out_china.write(line)
                    else:
                        f_out_final.write(line)
            else:
                f_out_date.write(line)
        else:
            f_out_date.write(line)


# 2.Worldseq 【global seqs】

## get seq names

In [2]:
# Extract sequence names through metadata

input_file = "2.filter_metadata/metadata_filter_china_date.tsv"
output_seq_name = "2.filter_metadata/world_seqname.tsv"

with open(input_file, 'r') as f_in, open(output_seq_name, 'w') as f_out_seqname:
    for line in f_in:
        columns = line.split('\t')
        f_out_seqname.write(columns[0] + "|" + columns[5] + "|" + columns[17] + "\n")

In [3]:
# stats
! wc -l 2.filter_metadata/world_seqname.tsv
! cut -f 6 2.filter_metadata/metadata_filter_china_date.tsv | sort | uniq

1868049 2.filter_metadata/filter_china_date_seqname.tsv
2022-10-01
2022-10-02
2022-10-03
2022-10-04
2022-10-05
2022-10-06
2022-10-07
2022-10-08
2022-10-09
2022-10-10
2022-10-11
2022-10-12
2022-10-13
2022-10-14
2022-10-15
2022-10-16
2022-10-17
2022-10-18
2022-10-19
2022-10-20
2022-10-21
2022-10-22
2022-10-23
2022-10-24
2022-10-25
2022-10-26
2022-10-27
2022-10-28
2022-10-29
2022-10-30
2022-10-31
2022-11-01
2022-11-02
2022-11-03
2022-11-04
2022-11-05
2022-11-06
2022-11-07
2022-11-08
2022-11-09
2022-11-10
2022-11-11
2022-11-12
2022-11-13
2022-11-14
2022-11-15
2022-11-16
2022-11-17
2022-11-18
2022-11-19
2022-11-20
2022-11-21
2022-11-22
2022-11-23
2022-11-24
2022-11-25
2022-11-26
2022-11-27
2022-11-28
2022-11-29
2022-11-30
2022-12-01
2022-12-02
2022-12-03
2022-12-04
2022-12-05
2022-12-06
2022-12-07
2022-12-08
2022-12-09
2022-12-10
2022-12-11
2022-12-12
2022-12-13
2022-12-14
2022-12-15
2022-12-16
2022-12-17
2022-12-18
2022-12-19
2022-12-20
2022-12-21
2022-12-22
2022-12-23
2022-12-24
2022-12-2

## get sequences.fasta

In [ ]:
! mkdir 3.filter_seq
! seqkit grep -f 2.filter_metadata/world_seqname.tsv 0.rawdata/seq.fil.fa > 3.filter_seq/world_seq.fasta

In [4]:
! seqkit stats 3.filter_seq/world_seq.fasta 

file                                       format  type   num_seqs         sum_len  min_len  avg_len  max_len
3.filter_seq/filter_china_date_seq2.fasta  FASTA   DNA   1,700,278  50,455,779,471   27,047   29,675   29,997


## Run nextclade pipeline

In [ ]:
# Nextclade
! mkdir 4.nextclade
import os
os.system("nohup nextclade run \
          --input-dataset /home/qiuming/sars-cov-2-0128update \
          --output-tsv=4.nextclade/world_nextclade.tsv \
          -j 48 \
          3.filter_seq/world_seq.fasta \
          > nohup.log 2>&1 &")

## filter sequences with unqualified quality and private mutations >= 10

In [1]:
# 4.filter bad & private.mutatution.number >= 10
count = 0
with open('4.nextclade/world_nextclade.tsv', 'r') as file, open('4.nextclade/world_nextclade_bad.tsv', 'w') as f_out_bad, open('4.nextclade/world_nextclade_good.tsv', 'w') as f_out_good:
    for line in file:
        if line.startswith('index'):
            continue
        columns = line.split('\t')
        
        if columns[9] == 'bad' \
            or columns[53] == 'bad' \
            or columns[57] == 'bad' \
            or (not columns[54]) \
            or (columns[54] and int(columns[54]) >= 10):
            count += 1
            f_out_bad.write(line)
        else:
            f_out_good.write(line)

print(count)

618843


In [10]:
# View clade distribution
! wc -l 4.nextclade/world_nextclade_good.tsv
! cut -f 6 4.nextclade/world_nextclade_good.tsv | sort | uniq -c

1081435 4.nextclade/output_good.tsv
     25 19A
      4 19B
      5 20A
      9 20B
      3 20C
      1 20J
      6 21J
     83 21K
   5948 21L
  12637 22A
 282890 22B
     16 22C
  49283 22D
 327337 22E
  42612 22F
 209597 23A
  27190 23B
  35789 23C
  53252 23D
  10260 23E
   5970 23F
    933 23G
     15 23H
     22 23I
  17548 recombinant


## extract final sequences.fasta

In [11]:
# output_good
! cut -f 2 4.nextclade/world_nextclade_good.tsv > 4.nextclade/world_nextclade_good_seqname.tsv
! seqkit grep -f 4.nextclade/world_nextclade_good_seqname.tsv 3.filter_seq/world_seq.fasta > 4.nextclade/world_good_seq.fasta

[INFO] 1078904 patterns loaded from file


In [3]:
! seqkit stats 4.nextclade/world_good_seq.fasta
! wc -l 4.nextclade/world_nextclade_good_seqname.tsv

file                              format  type   num_seqs         sum_len  min_len   avg_len  max_len
4.nextclade/world_good_seq.fasta  FASTA   DNA   1,081,435  32,085,632,198   27,264  29,669.5   29,991
1081435 4.nextclade/world_nextclade_good_seqname.tsv


## Randomly select 500 sequences to construct a phylogenetic tree🌲

In [ ]:
# Randomly select 500 sequences
! mkdir 5.random_500
! seqkit sample -n 500 -o 5.random_500/world_500.fasta 4.nextclade/world_good_seq.fasta
! seqkit seq -n 5.random_500/world_500.fasta > 5.random_500/world_500_random_500sequence_name.tsv

In [17]:
# acquire metadata
! awk -F'\t' '{print $1 "\t" $5 "\t" $6 "\t" $7}' 2.filter_metadata/metadata_filter_china_date.tsv > 2.filter_metadata/metadata_filter_china_date_clade.tsv


In [2]:
# output-good : run nextclade
! nextclade run \
    --input-dataset /home/qiuming/sars-cov-2-0128update \
    --output-tsv=5.random_500/world_500_nextclade.tsv \
    -j 48 \
    5.random_500/world_500.fasta

In [19]:
# modify seq names
! awk -F'\t' '{print $2 "\t" $3 "\t" $4}' 5.random_500/world_500_nextclade.tsv > 5.random_500/world_500_clade.tsv

with open('5.random_500/world_500_clade.tsv', 'r') as file, open('5.random_500/world_500_clade_match.tsv', 'w') as f_out:
    for line in file:
        columns = line.strip().split('\t')
        accession = columns[0].split('|')[0]
        columns.insert(0, accession)
        
        f_out.write('\t'.join(columns) + '\n')

In [20]:
# extract metadata
import pandas as pd

seqMeta = pd.read_csv('2.filter_metadata/metadata_filter_china_date_clade.tsv', sep='\t', header=None)
random_sequence = pd.read_csv('5.random_500/world_500_clade_match.tsv', sep='\t', header=None)

merged_data = pd.merge(random_sequence, seqMeta, left_on=0, right_on=0)
merged_data.to_csv('5.random_500/world_500_metadata.tsv', sep='\t', index=False, header=False)


# 3.ChinaSeq【chinese seqs】

## get seq names

In [1]:
# extract seq_names

input_file = "2.filter_metadata/metadata_china_unmatched.tsv"
output_seq_name = "2.filter_metadata/china_seqname.tsv"

with open(input_file, 'r') as f_in, open(output_seq_name, 'w') as f_out_seqname:
    for line in f_in:
        columns = line.split('\t')
        f_out_seqname.write(columns[0] + "|" + columns[5] + "|" + columns[17] + "\n")

## get sequences.fasta

In [ ]:
! mkdir 3.filter_seq
! seqkit grep -f 2.filter_metadata/china_seqname.tsv 0.rawdata/seq.fil.fa > 3.filter_seq/china_seq.fasta

In [3]:
# stats
! seqkit stats 3filter_seq/china_seq.fasta

file                          format  type  num_seqs        sum_len  min_len   avg_len  max_len
3.filter_seq/china_seq.fasta  FASTA   DNA     49,007  1,457,788,124   28,485  29,746.5   29,932


## Run nextclade pipeline

In [ ]:
# nextclade
! mkdir 4.nextclade
import os
os.system("nohup nextclade run \
        --input-dataset /home/qiuming/sars-cov-2-0128update \
        --output-tsv=4.nextclade/china_nextclade.tsv \
        -j 48 \
        3.filter_seq/china_seq.fasta \
        > nohup.log 2>&1 &")

## Filter sequences with unqualified quality and private mutations >= 10

In [5]:
# filter bad & private.mutatution.number >= 10
count = 0
with open('4.nextclade/china_nextclade.tsv', 'r') as file, open('4.nextclade/china_nextclade_bad.tsv', 'w') as f_out_bad, open('4.nextclade/china_nextclade_good.tsv', 'w') as f_out_good:
    for line in file:
        if line.startswith('index'):
            continue
        columns = line.split('\t')
        
        if columns[9] == 'bad' \
            or columns[53] == 'bad' \
            or columns[57] == 'bad' \
            or (not columns[54]) or (columns[54] and int(columns[54]) >= 10):
            count += 1
            f_out_bad.write(line)
        else:
            f_out_good.write(line)

print(count)

12065


In [7]:
## View clade distribution
! wc -l 4.nextclade/china_nextclade_good.tsv
! cut -f 6 4.nextclade/china_nextclade_good.tsv | sort | uniq -c

36942 4.nextclade/china_nextclade_good.tsv
     17 21L
      3 22A
  17613 22B
      5 22C
    761 22D
    190 22E
   2958 22F
   1563 23A
   4319 23B
     24 23C
   5640 23D
    392 23E
   3332 23F
      4 23G
     58 23H
     63 recombinant


## extract final sequences.fasta

In [9]:
# 提final sequences
! cut -f 2 4.nextclade/china_nextclade_good.tsv > 4.nextclade/china_nextclade_good_seqname.tsv
! seqkit grep -f 4.nextclade/china_nextclade_good_seqname.tsv 3.filter_seq/china_seq.fasta > 4.nextclade/china_good_seq.fasta

[INFO] 36942 patterns loaded from file


In [4]:
! seqkit stats 4.nextclade/china_good_seq.fasta
! wc -l 4.nextclade/china_nextclade_good_seqname.tsv

file                              format  type  num_seqs        sum_len  min_len   avg_len  max_len
4.nextclade/china_good_seq.fasta  FASTA   DNA     36,942  1,098,715,052   28,536  29,741.6   29,921
36942 4.nextclade/china_nextclade_good_seqname.tsv
